In [1]:
from random import random
import numpy as np
from numpy import array
from numpy import cumsum
from keras.models import Sequential
from keras import layers
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix

import pickle
from keras.models import Model
import os
os.environ["CUDA_VISIBLE_DEVICES"]="6"


Using TensorFlow backend.


In [ ]:
DATA = "ARTINT"

In [2]:
!nvidia-smi

Sun Jan 13 15:23:34 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 384.111                Driver Version: 384.111                   |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:04:00.0  On |                  N/A |
| 43%   67C    P2    77W / 250W |   9081MiB / 11172MiB |     12%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:05:00.0 Off |                  N/A |
| 23%   37C    P8    16W / 250W |    506MiB / 11172MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [3]:
train_filename = pickle.load(open(DATA+"_train_abstract_filename.p", "rb"))
train_use_vector = pickle.load(open(DATA+"_train_abstract_use_vector.p", "rb"))
test_filename = pickle.load(open(DATA+"_test_abstract_filename.p", "rb"))
test_use_vector = pickle.load(open(DATA+"_test_abstract_use_vector.p", "rb"))


hi


In [4]:
y = []
c = 0
for i in train_filename:
    c = c + 1
    if c%500 == 0:
        print(c)
    if "pos_" in i:
        y.append(1.0)
    else :
        y.append(0.0)

500
1000
1500
2000
2500
3000


In [5]:
y_test = []
c = 0
for i in test_filename:
    c = c + 1
    if c%500 == 0:
        print(c)
    if "pos_" in i:
        y_test.append(1.0)
    else :
        y_test.append(0.0)

500


In [6]:
print(train_use_vector.shape)
print(test_use_vector.shape)

(3056, 25, 512)
(768, 25, 512)


In [7]:

n_timesteps = train_use_vector[0].shape[0]
# define LSTM
model = Sequential()
model.add(Bidirectional(LSTM(100, return_sequences=True), input_shape=(n_timesteps, 512)))
model.add(Bidirectional(LSTM(50, return_sequences=True)))
model.add(Bidirectional(LSTM(50)))
model.add(layers.Dense(50, activation = "relu"))
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(100, activation = "relu"))
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(100, activation = "relu"))
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))

# Output- Layer
model.add(layers.Dense(1, activation = "sigmoid"))

model.summary()
checkpoint = ModelCheckpoint(DATA+'model-ABSTRACT-best.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='auto')  

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 25, 200)           490400    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 25, 100)           100400    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 100)               60400     
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               5100      
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
__________

In [8]:
results = model.fit( train_use_vector,y, epochs= 10, batch_size = 64,validation_data=(test_use_vector,y_test),callbacks=[checkpoint])


Train on 3056 samples, validate on 768 samples
Epoch 1/10
3056/3056 [==============================] - 20s 6ms/step - loss: 0.5405 - acc: 0.7817 - val_loss: 0.4146 - val_acc: 0.8294

Epoch 00001: val_loss improved from inf to 0.41458, saving model to model-best.h5
Epoch 2/10
3056/3056 [==============================] - 12s 4ms/step - loss: 0.3760 - acc: 0.8537 - val_loss: 0.3413 - val_acc: 0.8529

Epoch 00002: val_loss improved from 0.41458 to 0.34132, saving model to model-best.h5
Epoch 3/10
3056/3056 [==============================] - 12s 4ms/step - loss: 0.3064 - acc: 0.8789 - val_loss: 0.2676 - val_acc: 0.8854

Epoch 00003: val_loss improved from 0.34132 to 0.26765, saving model to model-best.h5
Epoch 4/10
3056/3056 [==============================] - 12s 4ms/step - loss: 0.2607 - acc: 0.8943 - val_loss: 0.2764 - val_acc: 0.8776

Epoch 00004: val_loss did not improve from 0.26765
Epoch 5/10
3056/3056 [==============================] - 12s 4ms/step - loss: 0.2392 - acc: 0.9077 - val_

In [9]:
model.layers

In [10]:
#model.load_weights('model-best.h5')

new_model = Model(inputs = model.inputs, outputs = model.layers[-3].output)

new_model.load_weights(DATA+'model-ABSTRACT-best.h5', by_name = True)
train_prediction = new_model.predict(train_use_vector)
test_prediction = new_model.predict(test_use_vector)

In [11]:
np.save(DATA+'_train_abstract_modality_feature',train_prediction)

np.save(DATA+'_test_abstract_modality_feature',test_prediction)


In [12]:
predicted =  model.predict(test_use_vector)
for i in range(len(predicted)):
    if predicted[i] >= 0.5:
        predicted[i] = 1.0
    else :
        predicted[i] = 0.0

In [13]:
print("fscore",f1_score(y_test, predicted, average="macro"))
print("precision",precision_score(y_test, predicted, average="macro"))
print("recall",recall_score(y_test, predicted, average="macro"))    

x = classification_report(y_test,predicted)
print(x)

fscore 0.9216624959199217
precision 0.9216624959199217
recall 0.9216624959199217
             precision    recall  f1-score   support

        0.0       0.92      0.92      0.92       364
        1.0       0.93      0.93      0.93       404

avg / total       0.92      0.92      0.92       768



In [14]:
model.save(DATA+'onlyabstractmodel')

In [15]:
pickle.dump(train_filename, open(DATA+"_train_abstract_filename.p", "wb"))  # save it into a file named save.p
pickle.dump(test_filename, open(DATA+"_test_abstract_filename.p", "wb"))  # save it into a file named save.p
